In [1]:
from lionagi import iModel, Branch

system_prompt = (
    "You are a hilarious dragon hunter who responds in 10 words rhymes"
)
hunter = Branch(name="hunter", system=system_prompt)

### Using OpenAI

to use openai, you need to obtain an OpenAI API key, you can get one [here](https://openai.com/api/)

1. save the key into your environment variables, either via a `.env` file or directly into your shell
2. the default key name we check is "OPENAI_API_KEY", you can specify which environment variable to use by setting the `api_key` parameter iModel or Service setup

In [2]:
# using openai's gpt-4o model

gpt4o = iModel(
    provider="openai",
    task="chat",  # chat/completions is the default endpoint
    model="gpt-4o",
    # api_key="OPENAI_API_KEY",     # change to "OPENAI_API_KEY_DEV" for example
)

In [3]:
# regular chat
a = await hunter.communicate(instruction="I am a dragon", imodel=gpt4o)

print(a)

Beware my spear, I'll roast you like a fiery tear!


### Using other endpoints

you can check the available tasks for an imodel using `list_tasks`

In [4]:
print(gpt4o.service.list_tasks())

['add_upload_part', 'cancel_batch', 'cancel_fine_tuning', 'cancel_upload', 'complete_upload', 'create_batch', 'create_chat_completion', 'create_embeddings', 'create_fine_tuning_job', 'create_image', 'create_image_edit', 'create_image_variation', 'create_moderation', 'create_speech', 'create_transcription', 'create_translation', 'create_upload', 'delete_file', 'delete_fine_tuned_model', 'list_batch', 'list_files', 'list_fine_tuning_checkpoints', 'list_fine_tuning_events', 'list_fine_tuning_jobs', 'list_models', 'retrieve_batch', 'retrieve_file', 'retrieve_file_content', 'retrieve_fine_tuning_job', 'retrieve_model', 'upload_file']


In [5]:
from pydantic import BaseModel


class Joke(BaseModel):
    joke: str

Structured output

In [6]:
b = await hunter.communicate(
    instruction="I am a dragon",
    imodel=gpt4o,
    response_format=Joke,
)

In [7]:
print(type(b))
b

<class '__main__.Joke'>


Joke(joke='With fiery claws, dragons hide their laughter flaws!')

### Structured output with Claude sonnet

LionAGI applies model-agnostic structured output, even if certain models don't provide built-in structured output we can have lionagi to take care of it.

similarly, you need to an anthropic api key, which can you obtain [here](https://www.anthropic.com/)

In [8]:
sonnet = iModel(
    provider="anthropic",
    model="claude-3-5-sonnet-20241022",
    max_tokens=1000,  # max_tokens is required for anthropic models
)

In [9]:
c = await hunter.communicate(
    instruction="I am a dragon",
    response_format=Joke,  # structured output in given pydantic model
    clear_messages=True,  # refresh the conversation
    imodel=sonnet,  # use sonnet model
)

In [10]:
c

Joke(joke="A dragon came by, so I yelled 'Time to fry!'")

### Query internet with Perplexity

you need to get an api key [here](https://docs.perplexity.ai/home)

In [11]:
pplx_small = iModel(
    provider="perplexity",
    task="chat/completions",
    model="llama-3.1-sonar-small-128k-online",
    max_tokens=1000,
)

In [12]:
b = await hunter.communicate(
    instruction="What makes a well-behaved dragon?",
    clear_messages=True,  # refresh the conversation
    imodel=pplx_small,  # use sonnet model
)

In [13]:
print(b)

A well-behaved dragon is one that's calm and bright,
No stress or fear, just a peaceful night.
It's active, not lethargic, with a happy face,
And behaviors like digging, not a frantic pace.
It's social, friendly, and never a fright,
Just a gentle soul, shining with delight


In [14]:
hunter.msgs.last_response.model_response

{'id': '1be10f4c-0936-4050-ab48-91bd86ab11a5',
 'model': 'llama-3.1-sonar-small-128k-online',
 'object': 'chat.completion',
 'created': 1734369700,
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': "A well-behaved dragon is one that's calm and bright,\nNo stress or fear, just a peaceful night.\nIt's active, not lethargic, with a happy face,\nAnd behaviors like digging, not a frantic pace.\nIt's social, friendly, and never a fright,\nJust a gentle soul, shining with delight"},
   'finish_reason': 'stop',
   'delta': {'role': 'assistant', 'content': ''}}],
 'usage': {'prompt_tokens': 40, 'completion_tokens': 69, 'total_tokens': 109},
 'citations': [{'url': 'https://dragonsdiet.com/blogs/dragon-care/15-bearded-dragon-behaviors-and-what-they-could-mean'},
  {'url': 'https://masterbraeokk.tripod.com/dragons/behavior.html'},
  {'url': 'https://files.eric.ed.gov/fulltext/ED247607.pdf'},
  {'url': 'https://www.travelchinaguide.com/intro/social_customs/zodiac/dragon/f